In [1]:
conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())

dense_model.add(layers.Dense(32, activation='relu'))
dense_model.add(laeyrs.BatchNormalization())

NameError: name 'conv_model' is not defined

In [2]:
from keras.models import Sequential, Model
from keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(layers.SeparableConv2D(32, 3, activation='relu', input_shape=(height, width, channels,)))
model.add(layers.SeparableConv2D(64, 2, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [7]:
#Using hyperas
from keras.datasets import imdb

import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

import numpy as np

from hyperopt import Trials, STATUS_OK, tpe
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils

from hyperas import optim
from hyperas.distributions import choice, uniform

def data():
    max_features = 2000
    max_len = 500

    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
    x_train = sequence.pad_sequences(x_train, maxlen=max_len)
    x_test = sequence.pad_sequences(x_test, maxlen=max_len)
    return x_train, y_train, x_test, y_test


def create_model(x_train, y_train, x_test, y_test):
    model = keras.models.Sequential()
    model.add(layers.Embedding(max_features, {{choice([128, 256, 512])}}, input_length=max_len, name='embed'))
    model.add(layers.Conv1D(32, {{choice([3, 5, 7])}}, activation='relu'))
    model.add(layers.MaxPooling1D({{choice([3, 5, 7])}}))
    model.add(layers.Conv1D(32, {{choice([3, 5, 7])}}, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(1))
    
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    
    result = model.fit(x_train, y_train, epochs={{quniform(2, 20)}}, batch_size={{choice([64, 128])}}, validation_split=0.2)
    
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=20,
                                          trials=Trials())
X_train, Y_train, X_test, Y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

OSError: [Errno 22] Invalid argument: 'C:\\Users\\bronto\\Desktop\\DeepLearning\\<ipython-input-7-631d2381b4bb>'